# Packages & Includes

In [1]:
using Plots
using StatsBase
using LinearAlgebra
using Statistics
using JLD2
using Dates
using LsqFit
using StaticArrays

# data structure module (needs to be module to avoid overwriting local variables in REPL scope)
include("SpinSimParams.jl")
using .SpinSimParams

# function libraries
include("spin_sims.jl");

# Basic Parameters

In [2]:
# interaction
α = 0.13;
ω = 0;

# spin ensemble
nfreq = 10000;

# I gave γ, τ, dt, ψ_0, bw, and ν0 default values since we almost never change them at this point

# lattice parameters
hlk = [1; 1];
θ = [π/2];

# wavevectors
ky = collect(exp10.(LinRange(-1, 1, 10)))
k = ();
for i = 1:size(ky,1)
    k = (k..., [0; ky[i]]);
end

# make the parameter file
params = make_params(α, ω, nfreq);

# load lattice parameters
params["hlk"] = hlk;
params["θ"] = θ;
params["k"] = k;

# create lattice
@time r, idx_r = make_lattice(params);

# set the variables to loop over
vars = ["k"];
params["vars"] = vars;
params["r"] = r;

# Run the Simulation (with spatial distribution)

In [4]:
I, d = make_idx(vars, params)
M_list = Array{Any}(undef, d)

for i in I

    # generate temporary parameters
    tparams = make_temp_params(params, vars, i)

    # simulate
    @time M_list[i] = spin_echo_sim(tparams)

end

# @save "001_M_list.jld2" M_list
# @save "001_parameters.jld2" params

 20.558463 seconds (369.64 M allocations: 20.919 GiB, 21.81% gc time)
 14.976529 seconds (366.65 M allocations: 20.772 GiB, 18.84% gc time)
 13.323031 seconds (366.65 M allocations: 20.772 GiB, 15.60% gc time)
 12.635837 seconds (366.65 M allocations: 20.772 GiB, 15.72% gc time)
 13.007604 seconds (366.65 M allocations: 20.772 GiB, 16.41% gc time)
 13.911760 seconds (366.65 M allocations: 20.772 GiB, 18.22% gc time)
 14.560105 seconds (366.65 M allocations: 20.772 GiB, 18.09% gc time)
 13.895571 seconds (366.65 M allocations: 20.772 GiB, 18.18% gc time)
 13.805082 seconds (366.65 M allocations: 20.772 GiB, 18.23% gc time)
 13.593003 seconds (366.65 M allocations: 20.772 GiB, 17.57% gc time)
